In [1]:
#To allocate memorey for gpu(here we are using 50%)
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [2]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator,load_img
from keras.models import Sequential

Using TensorFlow backend.


In [3]:
import numpy as np
from glob import glob

In [4]:
#resize all images
IMAGE_SIZE = [224,224]
train_path = 'Train'
test_path = 'Test'

In [5]:
vgg16 = VGG16(input_shape=IMAGE_SIZE + [3],weights='imagenet',include_top=False)

In [6]:
#dont train existing weights
for layer in vgg16.layers:
    layer.trainable = False

In [7]:
train_folders = glob('train/*')
train_folders

['train\\Birds', 'train\\Drones']

In [8]:
test_folders = glob('test/*')
test_folders

['test\\Birds', 'test\\Drones']

In [9]:
#flattern layer
x = Flatten()(vgg16.output)

In [18]:
prediction = Dense(2,activation='softmax')(x)
#create model
model = Model(inputs=vgg16.input,outputs=prediction)

In [11]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [19]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [20]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip = True)

In [21]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [22]:
training_set = train_datagen.flow_from_directory('Train',
                                                 target_size=(224,224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 660 images belonging to 2 classes.


In [23]:
test_set = test_datagen.flow_from_directory('Test',
                                            target_size = (224,224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 166 images belonging to 2 classes.


In [24]:
#fit the model
r = model.fit_generator(training_set,
                        validation_data = test_set,
                        epochs=20,
                        steps_per_epoch = len(training_set),
                        validation_steps = len(test_set))

Epoch 1/20
21/21 [==============================] - 111s 5s/step - loss: 0.5561 - accuracy: 0.7364 - val_loss: 0.2034 - val_accuracy: 0.9157
Epoch 2/20
21/21 [==============================] - 72s 3s/step - loss: 0.2519 - accuracy: 0.9076 - val_loss: 0.2510 - val_accuracy: 0.8976
Epoch 3/20
21/21 [==============================] - 72s 3s/step - loss: 0.2048 - accuracy: 0.9091 - val_loss: 0.0532 - val_accuracy: 0.9578
Epoch 4/20
21/21 [==============================] - 72s 3s/step - loss: 0.1709 - accuracy: 0.9318 - val_loss: 0.0297 - val_accuracy: 0.9398
Epoch 5/20
21/21 [==============================] - 71s 3s/step - loss: 0.1753 - accuracy: 0.9258 - val_loss: 0.2457 - val_accuracy: 0.9458
Epoch 6/20
21/21 [==============================] - 71s 3s/step - loss: 0.1245 - accuracy: 0.9591 - val_loss: 0.0405 - val_accuracy: 0.9759
Epoch 7/20
21/21 [==============================] - 71s 3s/step - loss: 0.1115 - accuracy: 0.9621 - val_loss: 0.0816 - val_accuracy: 0.9458
Epoch 8/20
21/21 [=

In [25]:
from keras.models import load_model
model.save('drone_bird_vgg16.h5')